<a href="https://colab.research.google.com/github/evriitt/style_transfer_bot/blob/main/art_style_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')
!pip install -r requirements.txt
!pip install telebot -q
import urllib
import telebot
import urllib.request
#import nest_asyncio
#nest_asyncio.apply()
!mkdir /content/user_image
!bash ./scripts/download_cyclegan_model.sh  style_monet
!bash ./scripts/download_cyclegan_model.sh  style_cezanne
!bash ./scripts/download_cyclegan_model.sh  style_ukiyoe
!bash ./scripts/download_cyclegan_model.sh  style_vangogh


In [2]:
TOKEN = ''

In [4]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def command_start(message):
    bot.send_message(
        message.chat.id,
        "Привет! Я умею преобразовывать изображение в один из четырех художественных стилей.\n"
        "Загрузи любое изображение через скрепку (прикрепить файл)"
    )

@bot.message_handler(content_types=['document'])
def scan_message(message):
    document_id = message.document.file_id
    file_info = bot.get_file(document_id)
    fi = file_info.file_path
    name = message.document.file_name
    urllib.request.urlretrieve(f'https://api.telegram.org/file/bot{TOKEN}/{fi}', f'/content/user_image/{name}')

    # Создаем клавиатуру с выбором стиля
    keyboard = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    keyboard.add("Клод Моне", "Поль Сезанн", "Укиё-э", "Ван Гог")
    bot.send_message(message.chat.id, "Теперь выбери стиль:", reply_markup=keyboard)

    # Ожидание ответа с выбором стиля
    bot.register_next_step_handler(message, process_style_selection, name)


def process_style_selection(message, name):
    style_choice = message.text
    bot.send_message(message.chat.id, "Cкоро вернусь")
    if style_choice == "Клод Моне":
        model_name = 'style_monet_pretrained'
    elif style_choice == "Поль Сезанн":
        model_name = 'style_cezanne_pretrained'
    elif style_choice == "Укиё-э":
        model_name = 'style_ukiyoe_pretrained'
    else:
        model_name = 'style_vangogh_pretrained'

    # Запускаем модель для обработки изображения с выбранным стилем
    command = f"python test.py --dataroot /content/user_image/ --name {model_name} --model test --no_dropout --gpu_ids -1 --preprocess scale_width --load_size 350"
    os.system(command)
    fake_img_path = f'/content/pytorch-CycleGAN-and-pix2pix/results/{model_name}/test_latest/images/{name[:-4]}_fake.png'

    # Отправляем обработанное изображение пользователю
    bot.send_message(message.chat.id, "Готово!")
    with open(fake_img_path, "rb") as photo_file:
        bot.send_photo(message.chat.id, photo_file)

    # Предложение пользователя выбрать другой стиль или загрузить новое изображение
    keyboard = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    keyboard.add("Другой стиль", "Загрузить новое изображение")
    bot.send_message(message.chat.id, "Что дальше?", reply_markup=keyboard)

    # Ожидание ответа пользователя
    bot.register_next_step_handler(message, handle_next_step, name)


def handle_next_step(message, name):
    choice = message.text
    if choice == "Другой стиль":
        keyboard = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        keyboard.add("Клод Моне", "Поль Сезанн", "Укиё-э", "Ван Гог")
        bot.send_message(message.chat.id, "Выбери другой стиль:", reply_markup=keyboard)
        bot.register_next_step_handler(message, process_style_selection, name)
    elif choice == "Загрузить новое изображение":
        bot.send_message(message.chat.id, "Загрузи новое изображение:")
        bot.register_next_step_handler(message, scan_message)


def main():
    bot.polling()

if __name__ == "__main__":
    main()
